# Implementing Flat Tax Rate
We will be looking at what people actually pay, what they should pay, and how to decrease to a flat tax rate. \
Evan Sellers + Michael Yager

In [129]:
import pandas as pd

In [355]:
df = pd.read_csv("./data/tax_data_2020.csv")
df = df[df.zipcode != 0]

In [131]:
def toMillion(amount):
    return round(amount / 1000000, 2)

def toBillion(amount):
    return round(amount / 1000000000, 2)

def toTrillion(amount):
    return round(amount / 1000000000000, 10)

## Percentage of Income Tax - Paided

In [149]:
# Remove Rows w/ Zero Income
df = df[df["A02650"] != 0]

In [184]:
df["taxPercentagePaid"] = df.apply(lambda row: row["A06500"] / row["A00100"], axis=1)

In [195]:
paid = df.groupby(["agi_stub"])["taxPercentagePaid"].mean()
paid

agi_stub
1    0.012746
2    0.044214
3    0.065613
4    0.078532
5    0.110051
6    0.188869
Name: taxPercentagePaid, dtype: float64

## Percentage of Income Tax - Expected

### Returns

In [188]:
df["taxPercentageReturn"] = df.apply(lambda row: 1 - (row["A00100"] / row["A02650"]), axis=1)

In [196]:
returns = df.groupby(["agi_stub"])["taxPercentageReturn"].mean()
returns

agi_stub
1    0.019301
2    0.012092
3    0.011414
4    0.009370
5    0.010816
6    0.010638
Name: taxPercentageReturn, dtype: float64

### Credits

In [191]:
TAX_CREDIT_AMT_1040  = [ "A07225", "A11070", "A10960", "A10970" ]
TAX_CREDIT_AMT_SCH3  = [ "A07230", "A07240", "A07180", "A07300", "A07260", "A09400", "A11450", "A11560" ]

In [192]:
def sumColumns(dataframe, columns):
    total = 0
    for col in columns:
        total += abs(dataframe[col])
    return total

In [202]:
df["credits"] = df.apply(lambda row: (sumColumns(row, TAX_CREDIT_AMT_SCH3) + sumColumns(row, TAX_CREDIT_AMT_1040)), axis=1)

In [203]:
df["taxPercentageCredit"] = df.apply(lambda row: row["credits"] / row["A02650"], axis=1)

In [206]:
df.groupby(["agi_stub"])["credits"].sum()

agi_stub
1    46161900.0
2    54919370.0
3    34827480.0
4    25014840.0
5    52657448.0
6    41222786.0
Name: credits, dtype: float64

In [204]:
credits = df.groupby(["agi_stub"])["taxPercentageCredit"].mean()
credits

agi_stub
1    0.065316
2    0.036898
3    0.025105
4    0.019558
5    0.014639
6    0.005525
Name: taxPercentageCredit, dtype: float64

This is intresting. This means lower income brackets take advantage of tax credits, and they are a larger impact to their overall taxes.

### Total

In [212]:
expected = paid + returns + credits
expected

agi_stub
1    0.097362
2    0.093203
3    0.102132
4    0.107461
5    0.135507
6    0.205032
dtype: float64

## Flat Tax Rate Basic
According to the congress report we have to account for `$1,609 Billion` in revenue for income taxes. \
[Revenues in Fiscal Year 2020](https://www.cbo.gov/system/files/2020-11/56746-MBR.pdf)

In [208]:
# Amount to acheive - reported by congress
print("$", toBillion(df2["A02650"].sum()*1000), "Billion")

$ 10743.66 Billion


In [209]:
# Percent each citizen must pay
1609/toBillion(df2["A02650"].sum()*1000)

0.1497627437949451

This number mean that most citizens must pay around `15%`, this is an issue though because this means most citizens are pay more than they currently are paying.

In [213]:
expected - paid

agi_stub
1    0.084617
2    0.048989
3    0.036520
4    0.028928
5    0.025455
6    0.016163
dtype: float64

This means according to the tax bracket that poorer individules are not paying their fair share according to the tax bracket.

## Flat Tax Rate Complexity
The issue is tax rate is not this simple. Because the tax brackets are progressive. So we must set a minimum that is taxed and a rate. 

In [237]:
df["rate"] = df.apply(lambda row: row["A06500"] / row["A00100"], axis=1)
PAID_RATE  = df.groupby("agi_stub")["rate"].mean()

In [253]:
def taxSystemRevenue(dataframe, minTaxable, rate):
    df["taxed"] = df.apply(lambda row: max((row["A02650"] - (row["N1"] * minTaxable)), 0) * rate, axis=1)
    taxed       = df.groupby("agi_stub")["taxed"].sum()
    income      = df.groupby("agi_stub")["A02650"].sum()
    rate        = (taxed/income).round(4)
    return rate, taxed.sum()

In [292]:
WEIGHTS = [2**6, 2**5, 2**4, 2**3, 2**2, 2]

def scoreTaxSystemRevenue(rate, revenue):
    if toBillion(revenue * 1000) < 1609:
        return 0
    diff  = rate - PAID_RATE
    wdiff = diff * WEIGHTS
    return wdiff.sum()

In [250]:
def simulateTaxSystemRevenue(dataframe, minTaxable, rate):
    rate, revenue = taxSystemRevenue(dataframe, minTaxable, rate)
    return scoreTaxSystemRevenue(rate, revenue)

### Search Grid

In [339]:
MIN_TAXABLE = list(range(0,100, 5))                                 # $0 - $100,000 by 1000s
TAX_RATE    = list(map(lambda x: round(x * 0.01, 2), range(1, 31))) # 0.01% - 0.30% by 0.01

### Perform Search

In [352]:
cols = list(map(lambda x: str(x) + "%", TAX_RATE))
rows = list(map(lambda x: "$" + "{:,}".format(x * 1000), MIN_TAXABLE))
grid = pd.DataFrame(columns=cols, index=rows)

In [ ]:
total = len(TAX_RATE) * len(MIN_TAXABLE)
done  = 0

for indexTaxRate in range(len(TAX_RATE)):
    results = []
    for indexMinTax in range(len(MIN_TAXABLE)):
        taxRate = TAX_RATE[indexTaxRate]
        minTax  = MIN_TAXABLE[indexMinTax]
        results.append(simulateTaxSystemRevenue(df, minTax, taxRate))
        done += 1
        print('\r' + str(done) + " of " + str(total), end="\r")
    grid[grid.columns[indexTaxRate]] = results